# 02 Data Quality Check — ASEAN Carbon Emission (2000–2024)
Notebook ini berfokus pada evaluasi kualitas dataset hasil preprocessing.
Pemeriksaan dilakukan terhadap missing values, duplikasi, dan coverage data per negara dan tahun.
Tujuan tahap ini adalah memberikan gambaran objektif kondisi data sebelum digunakan untuk EDA dan analisis lanjutan.

In [10]:
import pandas as pd
import numpy as np


## Load Data
Dataset hasil olahan dari notebook 01 dipanggil dari folder data/process. 
Langkah ini dilakukan agar proses analisis tidak mengulang download data mentah dan filter dari awal.

In [11]:
df = pd.read_csv("data/process/owid_co2_asean_2000_2024.csv")
df.head()


,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Brunei,2000,BRN,326429.0,NaN,0.0,0.0,5.886,-0.092,-1.537,...,NaN,0.017,0.0,0.0,0.0,0.0,9.218,8.123,-2.344,-39.830
1,Brunei,2001,BRN,333353.0,NaN,0.0,0.0,5.758,-0.128,-2.178,...,NaN,0.017,0.0,0.0,0.0,0.0,9.554,8.428,-2.237,-38.852
2,Brunei,2002,BRN,340108.0,NaN,0.0,0.0,5.285,-0.473,-8.206,...,NaN,0.017,0.0,0.0,0.0,0.0,8.517,7.424,-1.717,-32.479
3,Brunei,2003,BRN,346650.0,NaN,0.0,0.0,6.140,0.854,16.162,...,NaN,0.018,0.0,0.0,0.0,0.0,9.600,8.381,-0.516,-8.412
4,Brunei,2004,BRN,352921.0,NaN,0.0,0.0,5.967,-0.173,-2.817,...,NaN,0.018,0.0,0.0,0.0,0.0,8.826,7.879,-0.508,-8.519


Dataset ini akan menjadi dasar untuk seluruh analisis pada notebook berikutnya.

Selanjutnya ukuran dataset diperiksa sebagai validasi awal.

In [5]:
df.shape


(6350, 79)

## Struktur dan Tipe Data
Informasi tipe data dan ringkasan missing value ditampilkan untuk overview awal.

In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6350 entries, 0 to 6349
Data columns (total 79 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   country                                    6350 non-null   object 
 1   year                                       6350 non-null   int64  
 2   iso_code                                   5450 non-null   object 
 3   population                                 5725 non-null   float64
 4   gdp                                        3782 non-null   float64
 5   cement_co2                                 5722 non-null   float64
 6   cement_co2_per_capita                      5622 non-null   float64
 7   co2                                        6175 non-null   float64
 8   co2_growth_abs                             5875 non-null   float64
 9   co2_growth_prct                            5833 non-null   float64
 10  co2_including_luc       

## Duplicate Check
Duplikasi berdasarkan kombinasi country dan year diperiksa karena unit observasi didefinisikan pada level tersebut.

In [7]:
dup_count = df.duplicated(["country", "year"]).sum()
dup_count


np.int64(0)

Nilai duplikasi bernilai nol, menandakan setiap observasi country-year bersifat unik.

## Missing Values Overview
Persentase missing value dihitung untuk setiap kolom.

In [8]:
missing_pct = (
    df.isnull()
    .mean()
    .mul(100)
    .round(2)
    .sort_values(ascending=False)
)
missing_pct


,0
other_co2_per_capita,76.38
cumulative_other_co2,75.59
other_industry_co2,75.59
share_global_other_co2,75.59
share_global_cumulative_other_co2,75.59
...,...
share_of_temperature_change_from_ghg,6.69
temperature_change_from_co2,6.69
co2,2.76
year,0.00


Hasil ini menunjukkan kolom apa saja yang memiliki missing value yang tinggi dan berpotensi bermasalah untuk dilakukan analisis.

## Missing Values per Negara
missing value dilakukan per negara untuk menghindari bias struktural yang tersembunyi.

In [9]:
missing_by_country = (
    df.groupby("country")
    .apply(lambda x: x.isnull().mean().mul(100).round(2))
)
missing_by_country


/tmp/ipython-input-2466134547.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.isnull().mean().mul(100).round(2))


,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0
Africa,0.0,0.0,100.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0
Africa (GCP),0.0,0.0,100.0,100.0,100.0,100.0,100.0,0.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Albania,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0
Algeria,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wallis and Futuna,0.0,0.0,0.0,0.0,100.0,4.0,4.0,0.0,0.0,0.0,...,100.0,0.0,100.0,0.0,0.0,100.0,100.0,100.0,100.0,100.0
World,0.0,0.0,100.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Yemen,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,100.0


Langkah ini penting karena missing rendah secara global bisa menyembunyikan missing ekstrem pada negara tertentu.

## Coverage Tahun per Negara
Coverage tahun diperiksa untuk memastikan konsistensi periode antar negara.

In [12]:
coverage = (
    df.groupby("country")["year"]
    .agg(["min", "max", "nunique"])
    .sort_values("nunique")
)
coverage


,min,max,nunique
country,,,
Brunei,2000,2024,25
Cambodia,2000,2024,25
Indonesia,2000,2024,25
Laos,2000,2024,25
Malaysia,2000,2024,25
Myanmar,2000,2024,25
Philippines,2000,2024,25
Singapore,2000,2024,25
Thailand,2000,2024,25


Hasil ini menunjukkan apakah seluruh negara memiliki cakupan tahun yang seimbang pada periode 2000–2024.

## Definisi Kolom Inti untuk Analisis
Kolom inti didefinisikan secara eksplisit agar tidak terjadi redefinisi diam diam di notebook berikutnya.

In [15]:
core_columns = [
    "country",
    "year",
    "population",
    "co2",
    "co2_per_capita",
    "coal_co2",
    "oil_co2",
    "gas_co2",
    "cement_co2",
    "flaring_co2"
]

missing_core = missing_pct[core_columns]
missing_core


,0
country,0.00
year,0.00
population,9.84
co2,2.76
co2_per_capita,9.06
coal_co2,38.24
oil_co2,8.17
gas_co2,43.21
cement_co2,9.89
flaring_co2,6.69


## Evaluasi Threshold Missing
Threshold missing 10 persen digunakan sebagai rule of thumb untuk eksplorasi awal.
Kolom di atas threshold diidentifikasi tanpa langsung dihapus.

In [16]:
threshold = 10

cols_above_threshold = missing_pct[missing_pct > threshold]
cols_below_threshold = missing_pct[missing_pct <= threshold]

cols_above_threshold, cols_below_threshold


(other_co2_per_capita                         76.38
 cumulative_other_co2                         75.59
 other_industry_co2                           75.59
 share_global_other_co2                       75.59
 share_global_cumulative_other_co2            75.59
 consumption_co2_per_gdp                      51.04
 consumption_co2_per_capita                   48.30
 trade_co2                                    47.51
 trade_co2_share                              47.51
 gas_co2_per_capita                           44.00
 consumption_co2                              43.73
 share_global_cumulative_gas_co2              43.21
 cumulative_gas_co2                           43.21
 gas_co2                                      43.21
 share_global_gas_co2                         43.21
 energy_per_gdp                               40.63
 gdp                                          40.44
 coal_co2_per_capita                          39.02
 cumulative_coal_co2                          38.24
 share_globa

Keputusan untuk mengecualikan kolom tetap ditunda ke tahap analisis, bukan dipaksakan di tahap quality check.

## Dataset Ringkas untuk EDA
Dataset tidak dimodifikasi pada tahap ini.
Notebook ini hanya menghasilkan informasi kualitas dan definisi kolom.
Sebagai validasi akhir, jumlah baris per negara ditampilkan.

In [17]:
df["country"].value_counts().sort_values()


,count
country,
Brunei,25
Cambodia,25
Indonesia,25
Laos,25
Malaysia,25
Myanmar,25
Philippines,25
Singapore,25
Thailand,25


Output dari notebook ini digunakan sebagai dasar transparan untuk EDA dan analisis lanjutan, tanpa klaim normatif tentang kelayakan data.